## Hubert Prediction

A notebook to explore using Hubert. Based partially off of our work on Perceptual Qualities (https://arxiv.org/abs/2310.02497)

In [ ]:
import os
import sys

# Uncomment the below if you have a CUDA device/multiple
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append("../")

import torch
import torchaudio

import IPython
import matplotlib.pyplot as plt
import seaborn as sns

from src.utils import *

# Importing Sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

import time


In [ ]:
print(torch.__version__)
print(torchaudio.__version__)

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

In [ ]:
bundle = torchaudio.pipelines.HUBERT_BASE
# bundle = torchaudio.pipelines.HUBERT_XLARGE


print("Sample Rate:", bundle.sample_rate)

In [ ]:
model = bundle.get_model().to(device)

In [ ]:
print(model.__class__)

In [ ]:
# Let's Load in a Specific File
SPEECH_FILE = "../data/pvqd/audio_clips/BL10_ENSS.wav"
IPython.display.Audio(SPEECH_FILE)

In [ ]:
waveform, sample_rate = torchaudio.load(SPEECH_FILE)
waveform = waveform.to(device)

if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

In [ ]:
with torch.inference_mode():
    features = model.extract_features(waveform)
    features = features[0]

In [ ]:
# fig, ax = plt.subplots(len(features), 1, figsize=(16, 4.3 * len(features)))
# for i, feats in enumerate(features):
#     ax[i].imshow(feats[0].cpu(), interpolation="nearest")
#     ax[i].set_title(f"Feature from transformer layer {i+1}")
#     ax[i].set_xlabel("Feature dimension")
#     ax[i].set_ylabel("Frame (time-axis)")
    
# fig.tight_layout()

In [ ]:
# Dimension is BxTxD 
sixth_layer = features[5]
sixth_layer.shape

## Let's Use HuBERT for Some Prediction

In [ ]:
## Load in the DataFrame
y_train = pd.read_csv('../data/pvqd/train_test_split/y_train.csv', index_col=0)
y_val = pd.read_csv('../data/pvqd/train_test_split/y_val.csv', index_col=0)

# Load in the Test Set below if you want it

y_train.head()

We want to predict the five perceptual qualities: Breathiness, Loudness, Pitch, Roughness, and Strain. First, we have to link the files with their labels. 

In [ ]:
# Linking the `File` column with the actual File Name
# NOTE: This code is inefficient--why is it inefficient and how can we make it faster?

data_path = "../data/pvqd/audio_clips/"
audio_files = os.listdir(data_path)
speaker_ids = [extract_speaker(audio_file) for audio_file in audio_files]

# Assertion to make sure speaker_ids matches y_train['File']
i = 0
for spk_id in y_train["File"]:
    try:
        assert spk_id in speaker_ids
    except:
        print(spk_id)
        i+=1

# Dictionary to Link Speaker ID to Audio File for O(1) access
speaker_file_dict = {}
for i in range(0, len(speaker_ids)):
    speaker_file_dict[speaker_ids[i]] = os.path.join(data_path, audio_files[i])

### What model and features should we use?

As we saw earlier, there's a lot of layers in HuBERT, and each one contains different information (https://arxiv.org/pdf/2211.03929.pdf). The choice between layer 3 or layer 8 could result in vastly different predictive performance. Then there's the question of how we should even process the features! Should we use the raw shape of Batch-Size x Time-Dim x Feature Dim, or should we do some aggregation and perform a simple linear regression on top of that?

When it comes to machine learning and modeling, there's no reason not to try them all if you have the time (provided you've done the proper train-validation-test split first!). If you don't have the time, you have to use your best judgement to decide which archietctures, objective functions, and feature representations to explore first. This intuition comes with time.

To start off with, let's use Scikit-Learn (https://scikit-learn.org/stable/) to run some simple regression models (Linear, Lasso, and Random Forest) on a naive averaged version of the sixth HuBERT layer. 

In [ ]:
# Process the features and turn it into a dataframe
# NOTE: Is this the most efficient way to do this? How can we refactor this code? Should we be storing all this data in memory?

X_train = []
X_val = []

# Construct X_train
start_time = time.time() # Let's do some simple profiling
for spk_id in y_train["File"]:
    audio_file = speaker_file_dict[spk_id]

    # Load in the Audio and 
    waveform, sample_rate = torchaudio.load(audio_file)
    waveform = waveform.to(device)

    if sample_rate != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

    # Extract the HuBERT layers like before
    with torch.inference_mode():
        features = model.extract_features(waveform)
        features = features[0]

    sixth_layer = features[5]

    # TODO: Take the the mean over the Time dimension
    # Link: https://pytorch.org/docs/stable/generated/torch.mean.html 



    # Scikit learn doesn't work with Tensors, so we load them onto the CPU 
    # and convert to Numpy
    features = sixth_layer_mean[0].cpu().numpy() 
    X_train.append(features)

X_train = np.array(X_train)
elapsed_time = time.time() - start_time
print("Finished building X_train in %s seconds" % round(elapsed_time, 2))

# Construct X_val
start_time = time.time() # Let's do some simple profiling
for spk_id in y_val["File"]:
    audio_file = speaker_file_dict[spk_id]

    # Load in the Audio and 
    waveform, sample_rate = torchaudio.load(audio_file)
    waveform = waveform.to(device)

    if sample_rate != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

    # Extract the HuBERT layers like before
    with torch.inference_mode():
        features = model.extract_features(waveform)
        features = features[0]

    sixth_layer = features[5]

    # TODO: Take the the mean over the Time dimension
    # Link: https://pytorch.org/docs/stable/generated/torch.mean.html 


    # Scikit learn doesn't work with Tensors, so we load them onto the CPU 
    # and convert to Numpy
    features = sixth_layer_mean[0].cpu().numpy() 
    X_val.append(features)

X_val = np.array(X_val)
elapsed_time = time.time() - start_time
print("Finished building X_val in %s seconds" % round(elapsed_time, 2))


### Let's train some models!

Sanity Check: Are you absolutely certain that the first row of X_train is associated with the first row of y_train?

Below, we're going to train a linear and lasso regression model. How do you think it will do? 

In [ ]:
## TODO: Follow the documentation from sklearn.linear_model.Linear_Regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) 
## to train a Linear and Lasso Regression
np.random.seed(42) # Set the seed so that we all get the same results--hopefully

# TODO: Create and train a Linear Regression Model called reg
reg = ???


In [ ]:
# TODO: Use Reg to Predict X_train and evaluate

preds = ???
assert preds.shape == y_val.drop(["File"], axis=1).shape # Sanity check assertion

# TODO: Calculate the Mean Squared Error with mean_squared_error--Wait, is this right?
rmse = ???
print("Linear Model RMSE %s" % rmse)

We took the error over averaged oer every PQ! Let's try computing it for each PQ individually

In [ ]:
## TODO: Calculate the RMSE Column-wise


### Let's now train a Lasso

Some question: What's a lasso regression? Why would we want to try a lasso regression?

In [ ]:
# TODO: Create and train a Lasso Regression Model called lasso_reg with alpha = 0.1
lasso_reg = ???


In [ ]:
# TODO: Use lasso_reg to Predict X_train and evaluate


In [ ]:
## TODO: Calculate the RMSE Column-wise


### Questions

How does linear regression vs. lasso regression perform, based on RMSE?

### Let's Try Random Forest Regression

Linear models are nice and all, but they're just that--linear. Speech is an incredibly complex phenomena that can't possibly represented by linear relationships (right?). Let's see how linear models compare against [random forests](https://en.wikipedia.org/wiki/Random_forest), a common ensemble method that is particularly handy at characterizing non-linear relationships.  

In [ ]:
# TODO: Train a random forest model using [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
# Have you noticed a pattern with Sklearn yet?


start_time = time.time()
rf_reg = ??? # TODO
end_time = time.time()
print("Finished Training RF in %s Seconds" % (end_time- start_time))

In [ ]:
# TODO: Use rf_reg to Predict X_train and evaluate

rf_preds = ???
assert rf_preds.shape == y_val.drop(["File"], axis=1).shape # Sanity check assertion

# Calculate the Mean Squared Error--Wait, is this right?
rmse = 
print("Random Forest Model RMSE %s" % rmse)

In [ ]:
## TODO: Calculate the RMSE Column-wise

### Takeaways and Discussion

So, which model performed the best? Can you tell? How might you further investigate to further guarantee that one model performs better than another? (Hint: Look into bootstrapping)

What happens if you vary all the different models we chose? Different hyperparameters, different aggregation (why just do the mean?), different layers, a different representation altogether?

In a lot of Machine Learning problems, not just in Speech, there are decisions after decisions to be made and varied. 

## One Such Exploration: What happens if we vary the layer we examine?

In [ ]:
# Process the features and turn it into a dataframe
# NOTE: Is this the most efficient way to do this? How can we refactor this code? Should we be storing all this data in memory?

X_trains = []
X_vals = []

# Construct X_train
start_time = time.time() # Let's do some simple profiling
for spk_id in y_train["File"]:
    audio_file = speaker_file_dict[spk_id]

    # Load in the Audio and 
    waveform, sample_rate = torchaudio.load(audio_file)
    waveform = waveform.to(device)

    if sample_rate != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

    # Extract the HuBERT layers like before
    with torch.inference_mode():
        features = model.extract_features(waveform)
        features = features[0]

    all_layer_means = [layer.mean(dim=1)[0].cpu().numpy() for layer in features]

    X_trains.append(all_layer_means)

    

X_trains = np.array(X_trains)
elapsed_time = time.time() - start_time
print("Finished building X_train in %s seconds" % round(elapsed_time, 2))

# Construct X_val
start_time = time.time() # Let's do some simple profiling
for spk_id in y_val["File"]:
    audio_file = speaker_file_dict[spk_id]

    # Load in the Audio and 
    waveform, sample_rate = torchaudio.load(audio_file)
    waveform = waveform.to(device)

    if sample_rate != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
    
    with torch.inference_mode():
        features = model.extract_features(waveform)
        features = features[0]

    all_layer_means = [layer.mean(dim=1)[0].cpu().numpy() for layer in features]

    X_vals.append(all_layer_means)

X_vals = np.array(X_vals)
elapsed_time = time.time() - start_time
print("Finished building X_val in %s seconds" % round(elapsed_time, 2))


In [ ]:
X_trains.shape, X_vals.shape

### For each layer, we're going to train a linear regression and see how the error changes

In [ ]:
layer_err_dict = {
    "layer": [],
    "avg_rmse": []
}

for layer_num in range(0, X_trains.shape[1]):
    X_train = X_trains[:, layer_num, :]
    X_val = X_vals[:, layer_num, :]

    reg = LinearRegression().fit(X_train, y_train.drop(["File"], axis=1))

    preds = reg.predict(X_val)
    assert preds.shape == y_val.drop(["File"], axis=1).shape # Sanity check assertion

    # Calculate the Mean Squared Error--Wait, is this right?
    rmse = mean_squared_error(y_val.drop(["File"], axis=1).values, preds, squared=False)
    
    layer_err_dict["layer"].append(layer_num)
    layer_err_dict["avg_rmse"].append(rmse)

layer_err_df = pd.DataFrame(layer_err_dict)

In [ ]:
sns.lineplot(data=layer_err_df, x="layer", y="avg_rmse")
plt.ylim([0, 20])
plt.title("Avg. RMSE over HuBERT Layer")

## Congrats! We just did research